#  iWildCam2022 Show and Extract
cf. https://www.kaggle.com/code/stpeteishii/iwildcam2021-make-images-extracted

In [ ]:
import numpy as np
import pandas as pd
import os
import json
import random
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import tensorflow as tf
import pylab as pl
from PIL import Image
import math

#from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.preprocessing.image import load_img, img_to_array
#from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

#from sklearn.metrics import classification_report, log_loss, accuracy_score
#from sklearn.model_selection import train_test_split

# Check Data Structure

In [ ]:
data_dir = '../input/iwildcam2022-fgvc9/train'
test_dir = '../input/iwildcam2022-fgvc9/test'

In [ ]:
sample=pd.read_csv('../input/iwildcam2022-fgvc9/metadata/metadata/train_sequence_counts.csv')
sample[0:3]

In [ ]:
with open('../input/iwildcam2022-fgvc9/metadata/metadata/iwildcam2022_train_annotations.json') as json_data:
    train_anno = json.load(json_data)
    print(train_anno.keys())

In [ ]:
with open('../input/iwildcam2022-fgvc9/metadata/metadata/iwildcam2022_test_information.json') as json_data:
    test_anno = json.load(json_data)
    print(test_anno.keys())

In [ ]:
with open('../input/iwildcam2022-fgvc9/metadata/metadata/iwildcam2022_mdv4_detections.json') as json_data:
    megadetect = json.load(json_data)
    print(megadetect.keys())

In [ ]:
with open('../input/iwildcam2022-fgvc9/metadata/metadata/gps_locations.json') as json_data:
    location = json.load(json_data)
    print(location.keys())

In [ ]:
df_detect = pd.DataFrame(megadetect["images"])
df_detect
#### YOLO info for train and test

In [ ]:
df_detect.loc[0,'detections'][0]['bbox']

In [ ]:
with open('../input/iwildcam2022-fgvc9/metadata/metadata/iwildcam2022_test_information.json') as json_data:
    test_info = json.load(json_data)
    print(test_info.keys())

In [ ]:
df_test_info=pd.DataFrame(test_info['images'])
df_test_info[0:3]

In [ ]:
df_anno=pd.DataFrame(train_anno['annotations'])
df_anno[0:3]

In [ ]:
df_cat = pd.DataFrame(train_anno["categories"])
df_cat[0:3]

In [ ]:
N=[]
for i in range(205):
    N+=[i]
    
cat_id=df_cat['id']
Name=df_cat['name']

NM2NU_mapping=dict(zip(Name,N))
NU2NM_mapping=dict(zip(N,Name))
NM2ID_mapping=dict(zip(Name,cat_id)) 
ID2NM_mapping=dict(zip(cat_id,Name))
NU2ID_mapping=dict(zip(N,cat_id))
ID2NU_mapping=dict(zip(cat_id,N))  

# Function to show image and YOLO rectangle

In [ ]:
# for train data

def draw_bbox(img_path):
    
    img_file=img_path.split('/')[-2:]
    img_id=img_path.split('/')[-1][0:-4]
    img=mpimg.imread(img_path)
    detects=df_detect[df_detect.file==os.path.join(img_file[0],img_file[1])].detections.tolist() 
    print(detects)
    print(len(detects[0]))   
    if detects==[[]]:
        return None
    else:
        anno=df_anno[df_anno.image_id==img_id]    ### name
        cat_id=anno.category_id
        name=ID2NM_mapping[int(cat_id)]
        _ = plt.figure(figsize = (10,10))
        _ = plt.axis('off')
        ax = plt.gca()
        ax.text(10,100,f'{name}',fontsize=20,color='fuchsia')
        for detect in detects[0]:
            print(detect)
            x0,y0,w0,h0=detect['bbox']    
            H,W,_=img.shape
            x=x0*W
            y=y0*H
            w=w0*W
            h=h0*H
            cat='animal' if detect['category']=="1" else 'human'
            bbox=patches.FancyBboxPatch((x,y),w,h,alpha=0.8,capstyle='projecting',edgecolor='fuchsia',facecolor="none")
            ax.text(x+1.5,y-8,f'{cat} {detect["conf"]}',fontsize=10,bbox=dict(facecolor='fuchsia',alpha=0.8,edgecolor="none"))
            ax.add_patch(bbox)

    _ = plt.imshow(img)

In [ ]:
# for test data

def draw_test_bbox(img_path):
    
    img_file=img_path.split('/')[-2:]
    img_id=img_path.split('/')[-1][0:-4]
    img=mpimg.imread(img_path)
    detects=df_detect[df_detect.file==os.path.join(img_file[0],img_file[1])].detections.tolist()
    print(detects)
    print(len(detects[0])) 
    if detects==[[]]:
        return None
    else:
        _ = plt.figure(figsize = (10,10))
        _ = plt.axis('off')
        ax = plt.gca()
        ax.text(10,100,f'test',fontsize=20,color='fuchsia')
        for detect in detects[0]:
            print(detect)
            x0,y0,w0,h0=detect['bbox']    
            H,W,_=img.shape
            x=x0*W
            y=y0*H
            w=w0*W
            h=h0*H
            cat='animal' if detect['category']=="1" else 'human'
            bbox=patches.FancyBboxPatch((x,y),w,h,alpha=0.8,capstyle='projecting',edgecolor='fuchsia',facecolor="none")
            ax.text(x+1.5,y-8,f'{cat} {detect["conf"]}',fontsize=10,bbox=dict(facecolor='fuchsia',alpha=0.8,edgecolor="none"))
            ax.add_patch(bbox)

    _ = plt.imshow(img)

In [ ]:
path0 = "../input/iwildcam2022-fgvc9/train/train/915dbc6c-21bc-11ea-a13a-137349068a90.jpg"
_ = plt.figure(figsize = (10,10))
_ = plt.axis('off')
_ = plt.imshow(mpimg.imread(path0)[100:-100])

In [ ]:
draw_bbox(path0)

# Function to extract rectangle image

In [ ]:
# for train data

objects=[]
objlabels=[]

def extract_obj(img_path, show=False):

    img_file=img_path.split('/')[-2:]
    img_id=img_path.split('/')[-1][0:-4]
    img = mpimg.imread(img_path)
    detects = df_detect[df_detect.file==os.path.join(img_file[0],img_file[1])].detections.tolist()
    print(detects)
    print(len(detects[0]))    
    if len(detects)>0:      ######     
        if detects==[[]]:
            return None
        else:
            anno=df_anno[df_anno.image_id==img_id]
            cat_id=anno.category_id
            name=ID2NM_mapping[int(cat_id)]
            for idx,detect in enumerate(detects[0]):
                x0,y0,w0,h0=detect['bbox']    
                H,W,_=img.shape
                if w0*W>100 and h0*H>100:           
                    x=math.floor(x0*W)
                    y=math.floor(y0*H)
                    w=math.floor(w0*W)
                    h=math.floor(h0*H)
                    obj=img[int(y):int(y+h),int(x):int(x+w)]
                    obj1 = Image.fromarray(np.uint8(obj))
                    obj2 = np.asarray(obj1.resize((32,32))) 
                    objects.append(obj2/255.0)
                    objlabels.append(name)
                    if show:
                        _ = plt.figure(figsize=(3,3))
                        _ = plt.xticks([])
                        _ = plt.yticks([])
                        _ = plt.imshow(obj)

    return None

In [ ]:
# for test data

tobjects=[]
testid=[]

def extract_test_obj(img_path, show=False):

    img_file=img_path.split('/')[-2:]
    img_id=img_path.split('/')[-1][0:-4]
    img = mpimg.imread(img_path)
    detects = df_detect[df_detect.file==os.path.join(img_file[0],img_file[1])].detections.tolist()
    print(detects)
    print(len(detects[0])) 
    if len(detects)>0:      ######   
        if detects==[[]]:
            return None
        else:
            for idx,detect in enumerate(detects[0]):
                x0,y0,w0,h0=detect['bbox']    
                H,W,_=img.shape
                if w0*W>100 and h0*H>100:
                    x=math.floor(x0*W)
                    y=math.floor(y0*H)
                    w=math.floor(w0*W)
                    h=math.floor(h0*H)
                    obj=img[int(y):int(y+h),int(x):int(x+w)]
                    obj1 = Image.fromarray(np.uint8(obj))
                    obj2 = np.asarray(obj1.resize((32,32))) 
                    tobjects.append(obj2/255.0)
                    testid.append(img_id)

                    if show:
                        _ = plt.figure(figsize=(3,3))
                        _ = plt.xticks([])
                        _ = plt.yticks([])
                        _ = plt.imshow(obj)
            
    return None

In [ ]:
path1='../input/iwildcam2022-fgvc9/train/train/915dbc6c-21bc-11ea-a13a-137349068a90.jpg'

In [ ]:
draw_bbox(path1)

In [ ]:
extract_obj(path1, show=True)

In [ ]:
path2='../input/iwildcam2022-fgvc9/test/test/8676fd72-21bc-11ea-a13a-137349068a90.jpg'

In [ ]:
draw_test_bbox(path2)

In [ ]:
extract_test_obj(path2, show=True)